In [ ]:
import pandas as pd
import json
import sqlite3
import os
import glob
from tqdm import tqdm

# Ana Proje Yolu (Senin Drive yapına göre)
BASE_PATH = '/content/drive/MyDrive/FSTurk'
os.chdir(BASE_PATH)

print(f"📂 Çalışma dizini: {os.getcwd()}")
print("✅ Gerekli kütüphaneler yüklendi.")

📂 Çalışma dizini: /content/drive/MyDrive/FSTurk
✅ Gerekli kütüphaneler yüklendi.


In [ ]:
#" ", -, . gibi karakterler için özelleşmiş ZEMBEREK VE KAIKKI FİLTRASYONU

import json
import os
import glob
from collections import defaultdict

# --- AYARLAR ---
BASE_PATH = '/content/drive/MyDrive/FSTurk'
ZEMBEREK_FILE = os.path.join(BASE_PATH, "turkish_processed_zemberek_lexicon.json")
KAIKKI_FOLDER = os.path.join(BASE_PATH, "kaikki_processing")
OUTPUT_FILE = os.path.join(BASE_PATH, "final_lexicon_atomic_base.json")

# TR -> EN MAPPING
TAG_MAPPING = {
    'ISIM': 'NOUN',     'NOUN': 'NOUN',
    'FIIL': 'VERB',     'VERB': 'VERB',
    'SIFAT': 'ADJ',     'ADJ': 'ADJ',
    'ZARF': 'ADV',      'ADV': 'ADV',
    'ZAMIR': 'PRON',    'PRON': 'PRON',
    'BAGLAC': 'CONJ',   'CONJ': 'CONJ', 'CCONJ': 'CONJ',
    'SAYI': 'NUM',      'NUM': 'NUM',
    'EDAT': 'ADP',      'ADP': 'ADP',
    'BELIRTEC': 'DET',  'DET': 'DET',
    'OZEL ISIM': 'NOUN', 'PROPN': 'NOUN',
    'UNK': 'OTHER',     'OTHER': 'OTHER'
}

def is_clean_atomic_word(word, tag):
    """
    Kelimenin FST için uygun, temiz ve atomik olup olmadığını kontrol eder.
    """
    word = word.strip()

    # 1. Boşluk Kontrolü (Multi-word)
    if ' ' in word: return False

    # 2. Tire Kontrolü (Compound words) -> "e-devlet" istemiyoruz
    if '-' in word: return False

    # 3. Uzunluk Kontrolü (Tek harf 'o' hariç genelde çöp olabilir ama şimdilik tutalım)
    if len(word) == 0: return False

    # 4. Sayı/Sembol Kontrolü (NUM kategorisi hariç)
    # Eğer kelime NUM değilse ama içinde rakam varsa (örn: "covid19") atalım mı?
    # Evet, temiz sözlük için atalım.
    if tag != 'NUM':
        # isalpha(): Sadece harf mi? (Türkçe karakterler ve şapkalar dahildir)
        # Ancak tire ve boşluk zaten yukarıda elendi.
        # Bazı kısaltmalar (T.C.) noktalı olabilir. FST genelde noktasız sever.
        if '.' in word: return False

        # İçinde rakam var mı? (Word123 -> Çöp)
        if any(char.isdigit() for char in word): return False

    return True

def create_pristine_lexicon():
    print("💎 PRISTINE (TERTEMİZ) SÖZLÜK OLUŞTURULUYOR...")
    print("   🚫 Filtreler: Boşluk, Tire (-), Nokta (.), Rakamlı Kelimeler")

    grouped_data = defaultdict(set)
    dropped_count = 0

    # --- 1. ZEMBEREK ---
    print(f"1️⃣ Zemberek taranıyor...")
    if os.path.exists(ZEMBEREK_FILE):
        with open(ZEMBEREK_FILE, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)

            # Veri listeyse
            if isinstance(raw_data, list):
                for item in raw_data:
                    if len(item) >= 2:
                        word = item[0]
                        tr_tag = item[1]
                        en_tag = TAG_MAPPING.get(tr_tag, 'OTHER')

                        if is_clean_atomic_word(word, en_tag):
                            grouped_data[en_tag].add(word.strip())
                        else:
                            dropped_count += 1

            # Veri sözlükse
            elif isinstance(raw_data, dict):
                for tag, words in raw_data.items():
                    en_tag = TAG_MAPPING.get(tag, 'OTHER')
                    for w in words:
                        if is_clean_atomic_word(w, en_tag):
                            grouped_data[en_tag].add(w.strip())
                        else:
                            dropped_count += 1

    # --- 2. KAIKKI ---
    print(f"2️⃣ Kaikki taranıyor...")
    jsonl_files = glob.glob(os.path.join(KAIKKI_FOLDER, "*.jsonl"))

    for filepath in jsonl_files:
        filename = os.path.basename(filepath)
        if filename == "tr-extract.jsonl": continue

        raw_tag = filename.replace('.jsonl', '')
        en_tag = TAG_MAPPING.get(raw_tag, raw_tag) # Varsayılan: dosya adı

        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    word = data.get('word')
                    if word:
                        if is_clean_atomic_word(word, en_tag):
                            grouped_data[en_tag].add(word.strip())
                        else:
                            dropped_count += 1
                except: continue

    # --- 3. KAYDETME ---
    print(f"3️⃣ Final dosya yazılıyor...")

    final_output = {}
    total_words = 0

    for tag, word_set in grouped_data.items():
        final_output[tag] = sorted(list(word_set))
        total_words += len(final_output[tag])
        print(f"   🔹 {tag}: {len(final_output[tag])} kelime")

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, ensure_ascii=False)

    print(f"\n✨ TERTEMİZ BAŞLANGIÇ HAZIR!")
    print(f"   🗑️ Filtrelenen (Çöp/Bileşik) Kelime: {dropped_count}")
    print(f"   💎 Saf Kelime Sayısı: {total_words}")
    print(f"   📂 Dosya: {OUTPUT_FILE}")

create_pristine_lexicon()

💎 PRISTINE (TERTEMİZ) SÖZLÜK OLUŞTURULUYOR...
   🚫 Filtreler: Boşluk, Tire (-), Nokta (.), Rakamlı Kelimeler
1️⃣ Zemberek taranıyor...
2️⃣ Kaikki taranıyor...
3️⃣ Final dosya yazılıyor...
   🔹 OTHER: 90758 kelime
   🔹 PRON: 135 kelime
   🔹 NOUN: 145538 kelime
   🔹 ADJ: 10831 kelime
   🔹 NUM: 11 kelime
   🔹 VERB: 68180 kelime
   🔹 CONJ: 52 kelime
   🔹 ADV: 1703 kelime

✨ TERTEMİZ BAŞLANGIÇ HAZIR!
   🗑️ Filtrelenen (Çöp/Bileşik) Kelime: 34980
   💎 Saf Kelime Sayısı: 317208
   📂 Dosya: /content/drive/MyDrive/FSTurk/final_lexicon_atomic_base.json


In [ ]:
#--AMAÇ WIKIMEDIA VERİLERİNİ STANZA için  TÜRLERİNE AYIRMA--
import json
import os
import re
from tqdm import tqdm

# --- AYARLAR ---
BASE_PATH = '/content/drive/MyDrive/FSTurk'
# Şu anki en güncel dosyan bu (Zemberek + Kaikki + Gruplanmış):
INPUT_LEXICON = os.path.join(BASE_PATH, "final_lexicon_grouped_english.json")
CORPUS_FILE = os.path.join(BASE_PATH, "wikimedia_data", "cleaned_corpus.txt")
OUTPUT_FILE = os.path.join(BASE_PATH, "contain_zemberek_kaikki_wikimedia_with_stanza.json")

def add_corpus_words_to_waiting_room():
    print("🚀 WIKIPEDIA TARAMASI BAŞLIYOR...")

    # 1. Mevcut Sözlüğü Yükle
    if not os.path.exists(INPUT_LEXICON):
        print("❌ Sözlük bulunamadı!")
        return

    print(f"1️⃣ Mevcut sözlük yükleniyor...")
    with open(INPUT_LEXICON, 'r', encoding='utf-8') as f:
        lexicon_data = json.load(f)

    # Hızlı kontrol için mevcut kelimelerin hepsini bir kümeye (set) alalım
    existing_words = set()
    for tag, word_list in lexicon_data.items():
        for w in word_list:
            existing_words.add(w)

    print(f"   ✅ Hafızada zaten {len(existing_words)} kelime var.")

    # 2. Corpus Dosyasını Tara
    print(f"2️⃣ Corpus dosyasından YENİ kelimeler aranıyor...")

    if not os.path.exists(CORPUS_FILE):
        print("❌ Corpus dosyası yok! Lütfen wikimedia_data klasörünü kontrol et.")
        return

    new_candidates = set()
    # Regex: Sadece Türkçe karakter içeren kelimeleri al (noktalama yok, sayı yok)
    word_pattern = re.compile(r"[a-zA-ZçÇğĞıİöÖşŞüÜ]+")

    with open(CORPUS_FILE, 'r', encoding='utf-8') as f:
        # Tqdm ile ilerleme çubuğu göster
        for line in tqdm(f, desc="Wiki Taranıyor"):
            words = word_pattern.findall(line)
            for w in words:
                # Küçük harfe çevir (Lexicon standardı için)
                w_clean = w.lower()

                # Kurallar:
                # 1. 2 harften uzun olsun (ve, de, ki gibi bağlaçlar zaten vardır)
                # 2. Zaten sözlükte OLMASIN
                if len(w_clean) > 2 and w_clean not in existing_words:
                    new_candidates.add(w_clean)

    print(f"   ✨ Wikipedia'dan {len(new_candidates)} adet YEPYENİ kelime bulundu!")

    # 3. Bunları 'OTHER' Kategorisine Ekle (Bekleme Odası)
    if 'OTHER' not in lexicon_data:
        lexicon_data['OTHER'] = []

    # Kümeyi listeye çevirip ekle
    lexicon_data['OTHER'].extend(list(new_candidates))

    # OTHER içindeki tekrarları temizle
    lexicon_data['OTHER'] = sorted(list(set(lexicon_data['OTHER'])))

    print(f"   📦 Yeni kelimeler 'OTHER' (Formatlanacaklar) listesine eklendi.")
    print(f"   🧐 Şu an 'OTHER' toplam boyutu: {len(lexicon_data['OTHER'])}")

    # 4. Kaydet
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(lexicon_data, f, ensure_ascii=False)

    print(f"\n💾 Dosya Kaydedildi: {OUTPUT_FILE}")
    print("👉 SIRADAKİ ADIM: Stanza kodunu bu dosya üzerinde çalıştırıp 'OTHER'ları eritmek!")

add_corpus_words_to_waiting_room()

🚀 WIKIPEDIA TARAMASI BAŞLIYOR...
1️⃣ Mevcut sözlük yükleniyor...
   ✅ Hafızada zaten 322359 kelime var.
2️⃣ Corpus dosyasından YENİ kelimeler aranıyor...


Wiki Taranıyor: 100000it [00:35, 2812.30it/s]


   ✨ Wikipedia'dan 1033121 adet YEPYENİ kelime bulundu!
   📦 Yeni kelimeler 'OTHER' (Formatlanacaklar) listesine eklendi.
   🧐 Şu an 'OTHER' toplam boyutu: 1129886

💾 Dosya Kaydedildi: /content/drive/MyDrive/FSTurk/contain_zemberek_kaikki_wikimedia_with_stanza.json
👉 SIRADAKİ ADIM: Stanza kodunu bu dosya üzerinde çalıştırıp 'OTHER'ları eritmek!


In [ ]:
#T4 SEÇTİĞİMDEN EMİN OLUN.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install stanza

In [1]:
import torch
print("GPU Var mı?:", torch.cuda.is_available())
try:
    print("GPU Adı:", torch.cuda.get_device_name(0))
    print("✅ GPU AKTİF! Devam edebilirsin.")
except:
    print("❌ GPU YOK! Ayarları kontrol et.")

GPU Var mı?: True
GPU Adı: Tesla T4
✅ GPU AKTİF! Devam edebilirsin.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 43.8 MB/s eta 0:00:00


In [4]:
import stanza
import json
import os
from tqdm import tqdm
import math
import torch

# --- AYARLAR ---
BASE_PATH = '/content/drive/MyDrive/FSTurk'
INPUT_FILE = os.path.join(BASE_PATH, "contain_zemberek_kaikki_wikimedia_with_stanza.json")
OUTPUT_FILE = os.path.join(BASE_PATH, "MASTER_FINAL_LEXICON.json")
TEMP_SAVE_FILE = os.path.join(BASE_PATH, "temp_stanza_progress.json")

def turbo_stanza_safe():
    # 1. GPU ZORUNLU KONTROL
    if not torch.cuda.is_available():
        raise Exception("❌ HATA: GPU algılanamadı! Runtime ayarlarından T4 GPU seçili mi?")

    print("✅ GPU Algılandı, Turbo Mod Başlatılıyor...")

    # 2. Stanza Yükle
    stanza.download('tr', processors='tokenize,pos', verbose=False)
    # use_gpu=True ZORUNLU
    nlp = stanza.Pipeline('tr', processors='tokenize,pos', use_gpu=True, verbose=False, tokenize_no_ssplit=True)

    # 3. Veri Yükle
    if not os.path.exists(INPUT_FILE):
        print("❌ Dosya bulunamadı!")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    full_other_list = data.get('OTHER', [])
    print(f"   📦 Toplam 'OTHER' Kelime Sayısı: {len(full_other_list)}")

    if len(full_other_list) == 0:
        print("   ✅ İşlenecek veri yok!")
        return

    # 4. Checkpoint Kontrolü
    processed_results = {}

    if os.path.exists(TEMP_SAVE_FILE):
        print("   📂 Kayıtlı ilerleme bulundu! Devam ediliyor...")
        with open(TEMP_SAVE_FILE, 'r', encoding='utf-8') as f:
            processed_results = json.load(f)

            # Zaten yapılanları küme (set) içine alıp hızla eleyelim
            done_set = set()
            for v_list in processed_results.values():
                done_set.update(v_list)

            # List comprehension ile filtrele
            full_other_list = [w for w in full_other_list if w not in done_set]
            print(f"   ⏩ {len(done_set)} kelime atlandı. Kalan: {len(full_other_list)}")

    else:
        processed_results = {
            'NOUN': [], 'VERB': [], 'ADJ': [], 'ADV': [],
            'PRON': [], 'CONJ': [], 'NUM': [], 'ADP': [], 'DET': []
        }

    # 5. BATCH PROCESSING
    BATCH_SIZE = 500
    total_batches = math.ceil(len(full_other_list) / BATCH_SIZE)

    print("   🔥 İşlem Başlıyor...")
    save_counter = 0

    # Tqdm ile ilerle
    for i in tqdm(range(0, len(full_other_list), BATCH_SIZE), total=total_batches, desc="Turbo GPU"):
        batch_words = full_other_list[i : i + BATCH_SIZE]

        # Kelimeleri birleştir (Stanza için tek metin)
        batch_text = "\n".join(batch_words)

        try:
            doc = nlp(batch_text)

            for sentence in doc.sentences:
                for word_obj in sentence.words:
                    if word_obj.upos in ['NOUN', 'PROPN']: t = 'NOUN'
                    elif word_obj.upos == 'VERB': t = 'VERB'
                    elif word_obj.upos == 'ADJ': t = 'ADJ'
                    elif word_obj.upos == 'ADV': t = 'ADV'
                    elif word_obj.upos == 'PRON': t = 'PRON'
                    elif word_obj.upos in ['CCONJ', 'SCONJ']: t = 'CONJ'
                    elif word_obj.upos == 'NUM': t = 'NUM'
                    elif word_obj.upos == 'ADP': t = 'ADP'
                    elif word_obj.upos == 'DET': t = 'DET'
                    else: t = 'UNK'

                    if t != 'UNK':
                        processed_results[t].append(word_obj.text)
        except Exception as e:
            # Hata olsa bile devam et, sadece o batch'i atla
            continue

        # KAYDETME (Her 20 batch - 10k kelime)
        save_counter += 1
        if save_counter % 20 == 0:
            with open(TEMP_SAVE_FILE, 'w', encoding='utf-8') as f:
                json.dump(processed_results, f, ensure_ascii=False)

    # 6. FİNAL
    print("\n   💾 Bitti! Kaydediliyor...")
    final_data = data.copy()
    final_data['OTHER'] = []

    for tag, new_words in processed_results.items():
        if tag in final_data:
            final_data[tag].extend(new_words)
            final_data[tag] = sorted(list(set(final_data[tag])))
        else:
            final_data[tag] = sorted(list(set(new_words)))

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False)

    if os.path.exists(TEMP_SAVE_FILE):
        os.remove(TEMP_SAVE_FILE)

    print(f"🏆 FİNAL DOSYA HAZIR: {OUTPUT_FILE}")

turbo_stanza_safe()

✅ GPU Algılandı, Turbo Mod Başlatılıyor...
   📦 Toplam 'OTHER' Kelime Sayısı: 1129886
   📂 Kayıtlı ilerleme bulundu! Devam ediliyor...
   ⏩ 39686 kelime atlandı. Kalan: 1091019
   🔥 İşlem Başlıyor...


Turbo GPU: 100%|██████████| 2183/2183 [56:42<00:00,  1.56s/it]



   💾 Bitti! Kaydediliyor...
🏆 FİNAL DOSYA HAZIR: /content/drive/MyDrive/FSTurk/MASTER_FINAL_LEXICON.json


In [5]:
import json
import random
import os

FILE_PATH = '/content/drive/MyDrive/FSTurk/MASTER_FINAL_LEXICON.json'

def check_lexicon_health():
    if not os.path.exists(FILE_PATH):
        print("❌ Dosya bulunamadı! Yolu kontrol et.")
        return

    print("📂 Dosya yükleniyor (Biraz sürebilir)...")
    with open(FILE_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"✅ Dosya başarıyla yüklendi!")
    print("="*40)

    total_words = 0

    # Her kategoriden rastgele 5 örnek göster
    for category, word_list in data.items():
        count = len(word_list)
        total_words += count
        print(f"🔹 {category:<10} : {count:,} kelime")

        # Örnekler
        if count > 0:
            sample = random.sample(word_list, min(5, count))
            print(f"   Örnekler: {', '.join(sample)}")
        print("-" * 20)

    print("="*40)
    print(f"📊 TOPLAM KELİME SAYISI: {total_words:,}")

check_lexicon_health()

📂 Dosya yükleniyor (Biraz sürebilir)...
✅ Dosya başarıyla yüklendi!
🔹 OTHER      : 0 kelime
--------------------
🔹 PRON       : 685 kelime
   Örnekler: benzeri, sizgen, şunla, birinede, kendilerinin
--------------------
🔹 NOUN       : 986,200 kelime
   Örnekler: relerin, lappe, canmore, gaviiformes, stanbulkarta
--------------------
🔹 ADJ        : 48,022 kelime
   Örnekler: klimatolojik, mücellâ, gotın, sevmeyenin, edepli
--------------------
🔹 NUM        : 67,259 kelime
   Örnekler: stivers, klavo, felâketzede, botanic, dakara
--------------------
🔹 VERB       : 207,486 kelime
   Örnekler: getirileceksin, başlamayan, dengeleneceğine, yükseltilebilen, uzayacaktı
--------------------
🔹 CONJ       : 253 kelime
   Örnekler: ille velâkin, kaldı ki, görüro, muuttaa, emde
--------------------
🔹 ADV        : 5,315 kelime
   Örnekler: taptaze, kız kıza, memnun, bedava, ciddiyken
--------------------
🔹 ADP        : 3,947 kelime
   Örnekler: kulluğumuz, musavve, milleluci, ağıdıyken, linde
-----

In [6]:
#GEREKSİZ KELİME TEMİZLİĞİ 1
import json
import re
import os
from collections import Counter

# Dosya Yolları
INPUT_FILE = '/content/drive/MyDrive/FSTurk/MASTER_FINAL_LEXICON.json'
OUTPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS.json'

def refinery_logic():
    print("🧹 Temizlik Operasyonu Başlıyor...")

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    clean_lexicon = {
        'NOUN': set(), 'VERB': set(), 'ADJ': set(),
        'ADV': set(), 'PRON': set(), 'CONJ': set(),
        'ADP': set(), 'DET': set(), 'NUM': set(), 'PROPN': set()
    }

    # İstemediğimiz karakterler
    garbage_pattern = re.compile(r'[wqx0-9_@\.\-\'\+]')

    # Fiil temizleyici (Basit Kural Tabanlı Stemmer)
    # Amacımız "getirileceksin" -> "getir" yapmak
    def clean_verb(word):
        # Sondan eklemeli dillerde tersten budama
        suffixes = [
            'mek', 'mak', 'me', 'ma', 'iyor', 'ıyor', 'uyor', 'üyor',
            'ecek', 'acak', 'mış', 'miş', 'muş', 'müş', 'dı', 'di', 'du', 'dü',
            'sın', 'sin', 'sun', 'sün', 'lar', 'ler', 'nız', 'niz'
        ]

        root = word
        # Basit bir döngüyle sondaki ekleri at (Çok agresif olmadan)
        # Kök en az 3 harf kalmalı (yememek -> ye)
        original = word
        for _ in range(3): # 3 katman ek silebilir
            for suf in suffixes:
                if root.endswith(suf) and len(root) > len(suf) + 2:
                    root = root[:-len(suf)]
                    break
        return root

    stats = {'deleted': 0, 'kept': 0, 'trimmed': 0}

    for category, words in data.items():
        print(f"   Süzülüyor: {category} ({len(words)} aday)...")

        for word in words:
            word = word.lower().strip()

            # 1. Kural: Çöp karakterler ve Uzunluk
            if garbage_pattern.search(word) or len(word) < 2 or len(word) > 12:
                stats['deleted'] += 1
                continue

            # 2. Kural: Yapışık Kelimeler (Parser Hataları)
            # "yapılmıştırbu" -> sonu 'bu' ile bitiyorsa ve uzunsa şüphelidir
            if (word.endswith('bu') or word.endswith('ve')) and len(word) > 8:
                stats['deleted'] += 1
                continue

            # 3. Kural: Kategoriye Özel Temizlik
            if category == 'VERB':
                root = clean_verb(word)
                if root != word:
                    stats['trimmed'] += 1
                clean_lexicon['VERB'].add(root)

            elif category == 'NOUN':
                # İsimlerde çok fazla çekim eki temizlemiyoruz, FST halletsin
                # Ama çok uzunları almayalım
                if len(word) < 10:
                    clean_lexicon['NOUN'].add(word)

            elif category == 'NUM':
                # Sayı olmayanları at (stivers vs.)
                # Yazıyla yazılan sayılar kalsın (bir, iki, yüz...)
                valid_nums = ["bir", "iki", "üç", "dört", "beş", "altı", "yedi", "sekiz", "dokuz", "on", "yirmi", "otuz", "kırk", "elli", "altmış", "yetmiş", "seksen", "doksan", "yüz", "bin", "milyon", "milyar", "sıfır"]
                if word in valid_nums:
                    clean_lexicon['NUM'].add(word)

            elif category == 'DET':
                # 'yayınlanmıştırbu' gibi hataları elemek için sadece bilinenleri alabiliriz
                # veya kısa olanları tutabiliriz
                if len(word) < 8:
                    clean_lexicon['DET'].add(word)

            else:
                clean_lexicon[category].add(word)

    # Listeye çevirip kaydet
    final_output = {k: sorted(list(v)) for k, v in clean_lexicon.items()}

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, ensure_ascii=False)

    print("\n✅ TEMİZLİK BİTTİ!")
    print(f"   🗑️ Silinen (Çöp/Hata): {stats['deleted']:,}")
    print(f"   ✂️ Budanan (Fiil Kökü): {stats['trimmed']:,}")

    total_clean = sum(len(v) for v in final_output.values())
    print(f"   💎 Kalan Saf Kelime: {total_clean:,}")
    print(f"   📂 Kayıt Yeri: {OUTPUT_FILE}")

    # VERB Örnekleri
    print(f"\n   Fiil Örnekleri (Kök Haline Dönmüş mü?):")
    print(f"   {final_output['VERB'][:10]}")

refinery_logic()

🧹 Temizlik Operasyonu Başlıyor...
   Süzülüyor: OTHER (0 aday)...
   Süzülüyor: PRON (685 aday)...
   Süzülüyor: NOUN (986200 aday)...
   Süzülüyor: ADJ (48022 aday)...
   Süzülüyor: NUM (67259 aday)...
   Süzülüyor: VERB (207486 aday)...
   Süzülüyor: CONJ (253 aday)...
   Süzülüyor: ADV (5315 aday)...
   Süzülüyor: ADP (3947 aday)...
   Süzülüyor: DET (505 aday)...

✅ TEMİZLİK BİTTİ!
   🗑️ Silinen (Çöp/Hata): 305,336
   ✂️ Budanan (Fiil Kökü): 42,839
   💎 Kalan Saf Kelime: 650,514
   📂 Kayıt Yeri: /content/drive/MyDrive/FSTurk/CLEAN_ROOTS.json

   Fiil Örnekleri (Kök Haline Dönmüş mü?):
   ['aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaa', 'aaaaaad', 'aaadır', 'aban', 'abanabil', 'abanabilir', 'abanacağım']


In [7]:
#GEREKSİZ EKLER TEMİZLİĞİ
import json
import re
import os

INPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS.json'
OUTPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS_V2.json'

def acid_bath_cleaning():
    print("🧪 Asit Banyosu Başlıyor (Agresif Temizlik)...")

    if not os.path.exists(INPUT_FILE):
        print("❌ Önceki dosya yok! Lütfen önce Rafineri 1'i çalıştır.")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    final_lexicon = {k: set() for k in data.keys()}

    # Türkçe Sesli Harfler
    vowels = set("aeıioöuü")
    # 3 tane yan yana aynı harf yakalayan regex (aaa, bbb)
    triple_char_regex = re.compile(r'(.)\1\1')

    # Agresif Fiil Budayıcı
    def aggressive_verb_stemmer(word):
        # Önce çekim ekleri (Tense/Person)
        inflectional_suffixes = [
            'casına', 'cesine', 'mı', 'mi', 'mu', 'mü',
            'tır', 'tir', 'tur', 'tür', 'dır', 'dir', 'dur', 'dür',
            'nız', 'niz', 'nuz', 'nüz', 'lar', 'ler', 'sın', 'sin', 'sun', 'sün',
            'ız', 'iz', 'uz', 'üz', 'ım', 'im', 'um', 'üm',
            'dı', 'di', 'du', 'dü', 'tı', 'ti', 'tu', 'tü',
            'mış', 'miş', 'muş', 'müş', 'yacak', 'yecek', 'acak', 'ecek',
            'yor', 'ıyor', 'iyor', 'uyor', 'üyor',
            'malı', 'meli', 'sa', 'se', 'a', 'e'
        ]

        # Sonra yapım/çatı ekleri (Derivational) - Sıra önemli!
        derivational_suffixes = [
            'abil', 'ebil', 'yabil', 'yebil', # Yeterlilik
            'iver', 'ıver', 'uver', 'üver',   # Tezlik
            'yaz', 'eyaz',                    # Yaklaşma
            'dur', 'edur',                    # Süerlik
            'mek', 'mak', 'me', 'ma'          # Mastar
        ]

        root = word

        # 1. Tur: Çekim eklerini at
        changed = True
        while changed:
            changed = False
            for suf in inflectional_suffixes:
                if root.endswith(suf) and len(root) > len(suf) + 2: # Kök en az 2 harf kalsın (ye, de)
                    root = root[:-len(suf)]
                    changed = True
                    break

        # 2. Tur: Çatı/Birleşik eklerini at (aban-abil -> aban)
        changed = True
        while changed:
            changed = False
            for suf in derivational_suffixes:
                if root.endswith(suf) and len(root) > len(suf) + 2:
                    root = root[:-len(suf)]
                    changed = True
                    break

        return root

    stats = {'deleted': 0, 'trimmed': 0}

    for category, word_list in data.items():
        print(f"   İşleniyor: {category} ({len(word_list)} kelime)...")

        for word in word_list:
            original_word = word

            # KURAL 1: 'aaaa' kontrolü
            if triple_char_regex.search(word):
                stats['deleted'] += 1
                continue

            # KURAL 2: Sesli harf kontrolü (Sessiz harflerden oluşan kısaltmaları at)
            if not any(char in vowels for char in word):
                stats['deleted'] += 1
                continue

            # KURAL 3: Çok uzun ve anlamsız kelimeler
            if len(word) > 15: # Türkçe kökler bu kadar uzun olmaz
                stats['deleted'] += 1
                continue

            # KURAL 4: Agresif Stemming
            if category == 'VERB':
                root = aggressive_verb_stemmer(word)
                if root != word:
                    stats['trimmed'] += 1
                final_lexicon['VERB'].add(root)
            elif category == 'NOUN':
                 # İsimlerde de basit çoğul eki temizliği yapalım
                 if word.endswith('lar') and len(word) > 5:
                     final_lexicon['NOUN'].add(word[:-3])
                     stats['trimmed'] += 1
                 elif word.endswith('ler') and len(word) > 5:
                     final_lexicon['NOUN'].add(word[:-3])
                     stats['trimmed'] += 1
                 else:
                     final_lexicon['NOUN'].add(word)
            else:
                final_lexicon[category].add(word)

    # Dosyayı kaydet
    output_data = {k: sorted(list(v)) for k, v in final_lexicon.items()}

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False)

    total_clean = sum(len(v) for v in output_data.values())

    print("\n🧪 ASİT BANYOSU TAMAMLANDI!")
    print(f"   🗑️ Eritilen (aaaa/çöp): {stats['deleted']:,}")
    print(f"   ✂️ Kesilen (aban-abil): {stats['trimmed']:,}")
    print(f"   💎 Kalan Saf Kök: {total_clean:,}")
    print(f"   📂 Yeni Dosya: {OUTPUT_FILE}")

    print(f"\n   Yeni Fiil Örnekleri (İlk 10):")
    print(f"   {output_data['VERB'][:10]}")

acid_bath_cleaning()

🧪 Asit Banyosu Başlıyor (Agresif Temizlik)...
   İşleniyor: NOUN (507171 kelime)...
   İşleniyor: VERB (93679 kelime)...
   İşleniyor: ADJ (40859 kelime)...
   İşleniyor: ADV (4507 kelime)...
   İşleniyor: PRON (571 kelime)...
   İşleniyor: CONJ (196 kelime)...
   İşleniyor: ADP (3471 kelime)...
   İşleniyor: DET (42 kelime)...
   İşleniyor: NUM (18 kelime)...
   İşleniyor: PROPN (0 kelime)...

🧪 ASİT BANYOSU TAMAMLANDI!
   🗑️ Eritilen (aaaa/çöp): 6,010
   ✂️ Kesilen (aban-abil): 52,463
   💎 Kalan Saf Kök: 613,755
   📂 Yeni Dosya: /content/drive/MyDrive/FSTurk/CLEAN_ROOTS_V2.json

   Yeni Fiil Örnekleri (İlk 10):
   ['aban', 'abanabilir', 'abanacağ', 'aband', 'abandık', 'abandın', 'abandırır', 'abanmışsı', 'abanozlaş', 'abanozlaşır']


In [9]:
import json
import os

INPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS_V2.json'
OUTPUT_FILE = '/content/drive/MyDrive/FSTurk/MASTER_FINAL_ROOTS.json'

def turbo_smart_reduction():
    print("🚀 TURBO İndirgeme Başlıyor (Set Lookup Strategy)...")

    if not os.path.exists(INPUT_FILE):
        print("❌ Dosya bulunamadı!")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Ek listesi (Aynı liste)
    valid_suffixes = tuple([ # Tuple yaptık ki startswith daha hızlı çalışsın
        'mak', 'mek', 'ma', 'me', 'ış', 'iş', 'uş', 'üş',
        'yor', 'yabil', 'yebil', 'yacak', 'yecek', 'r', 'ar', 'er', 'ır', 'ir', 'ur', 'ür',
        'dı', 'di', 'du', 'dü', 'tı', 'ti', 'tu', 'tü', 'mış', 'miş', 'muş', 'müş',
        'sın', 'sin', 'sun', 'sün', 'nız', 'niz', 'k', 'n', 'm',
        'abil', 'ebil', 'iver', 'ıver', 'dur', 'calama', 'celeme',
        'lar', 'ler', 'nın', 'nin', 'nun', 'nün', 'dan', 'den', 'tan', 'ten',
        'da', 'de', 'ta', 'te', 'ya', 'ye', 'yı', 'yi', 'yu', 'yü',
        'lı', 'li', 'lu', 'lü', 'sız', 'siz', 'suz', 'süz', 'lık', 'lik', 'luk', 'lük',
        'cı', 'ci', 'cu', 'cü', 'ça', 'çe', 'ca', 'ce', 'ki', 'kü', 'sı', 'si',
        'ı', 'i', 'u', 'ü' # Belirtme hali eklerini de ekledim
    ])

    final_lexicon = {}
    stats = {'deleted': 0, 'kept': 0}

    for category, word_list in data.items():
        print(f"   ⚡ Hızlandırılmış Tarama: {category} ({len(word_list)} kelime)...")

        # 1. Kelimeleri yine kısadan uzuna sırala
        sorted_words = sorted(list(word_list), key=len)

        # SET kullanıyoruz (Arama hızı O(1))
        kept_roots = set()

        category_deleted = 0

        for word in sorted_words:
            # Çok kısa kelimeleri direkt al
            if len(word) <= 3:
                kept_roots.add(word)
                continue

            is_derivative = False

            # YENİ MANTIK: Kelimenin kendisi içinde "prefix" arıyoruz
            # word = "kitaplık" -> i=3'ten başla
            # kitaplık -> 'kit', 'kita', 'kitap'...

            # Kelimenin uzunluğu kadar döngü (Max 15-20 tur, 1 Milyon tur değil!)
            for i in range(3, len(word)):
                prefix = word[:i] # Potansiyel kök
                suffix = word[i:] # Geriye kalan

                # 1. Bu prefix bizim elimizdeki köklerde var mı?
                if prefix in kept_roots:
                    # 2. Geriye kalan kısım geçerli bir ek mi?
                    if suffix.startswith(valid_suffixes):
                        is_derivative = True
                        break

            if is_derivative:
                category_deleted += 1
                stats['deleted'] += 1
            else:
                kept_roots.add(word)

        final_lexicon[category] = sorted(list(kept_roots))
        print(f"      -> {category} bitti. {category_deleted} kelime elendi.")

    # Dosyayı kaydet
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_lexicon, f, ensure_ascii=False)

    total = sum(len(v) for v in final_lexicon.values())

    print("\n🏆 TURBO OPERASYON TAMAMLANDI!")
    print(f"   🗑️ Toplam Elenen: {stats['deleted']:,}")
    print(f"   💎 Kalan Saf Kökler: {total:,}")
    print(f"   📂 Dosya: {OUTPUT_FILE}")

    # Kontrol
    if 'VERB' in final_lexicon:
        sample = [w for w in final_lexicon['VERB'] if w.startswith('aban')]
        print(f"\n   'aban' Kontrolü: {sample}")

turbo_smart_reduction()

🚀 TURBO İndirgeme Başlıyor (Set Lookup Strategy)...
   ⚡ Hızlandırılmış Tarama: NOUN (492050 kelime)...
      -> NOUN bitti. 348169 kelime elendi.
   ⚡ Hızlandırılmış Tarama: VERB (72152 kelime)...
      -> VERB bitti. 54776 kelime elendi.
   ⚡ Hızlandırılmış Tarama: ADJ (40782 kelime)...
      -> ADJ bitti. 12758 kelime elendi.
   ⚡ Hızlandırılmış Tarama: ADV (4502 kelime)...
      -> ADV bitti. 608 kelime elendi.
   ⚡ Hızlandırılmış Tarama: PRON (571 kelime)...
      -> PRON bitti. 385 kelime elendi.
   ⚡ Hızlandırılmış Tarama: CONJ (193 kelime)...
      -> CONJ bitti. 8 kelime elendi.
   ⚡ Hızlandırılmış Tarama: ADP (3445 kelime)...
      -> ADP bitti. 619 kelime elendi.
   ⚡ Hızlandırılmış Tarama: DET (42 kelime)...
      -> DET bitti. 4 kelime elendi.
   ⚡ Hızlandırılmış Tarama: NUM (18 kelime)...
      -> NUM bitti. 0 kelime elendi.
   ⚡ Hızlandırılmış Tarama: PROPN (0 kelime)...
      -> PROPN bitti. 0 kelime elendi.

🏆 TURBO OPERASYON TAMAMLANDI!
   🗑️ Toplam Elenen: 417,327
  